In [67]:
import matplotlib.pyplot as plt
import numpy as np
import torch
from mpl_toolkits.mplot3d import Axes3D

## Linear regression

$$ y_i = x_{ij} w_j + b$$

$$ y_i = x_{ij} w_j, \quad x_{i,-1}=1,\quad b=w_{-1} $$

In [68]:
def linear(x,w):
    return x @ w

Generate a random feature vector $\mathbf{x}$ witch 10000 samples and three feature 
such that first feature is drawn from N(0,1), second feature from  U(,1) and third from N(1,2).

N(mu,sigma) denotes normal distribution with mean mu and standard deviation sigma. You can use ``numpy.random.normal`` and ``numpy.random.uniform`` functions.

Using $\mathbf{x}$ and weights w = [0.2, 0.5,-0.25,1.0] generate output $\mathbf{y}$ assuming a $N(0,0.1)$ noise $\mathbf{\epsilon}$. 

$$ y_i = x_{ij} w_j+\epsilon_i, \quad x_{i,-1}=1,\quad b=w_{-1} $$

#### Loss

$$ \frac{1}{2}\frac{1}{N}\sum_{i=0}^{N-1} (y_i -  x_{ij} w_j  )^2$$

In [69]:
#Generate data    
plt.rcParams['figure.figsize'] = (12.0, 9.0)
n = 10000
x1 = np.random.normal(0, 1, n)
x2 = np.random.normal(0, 1, n)
x3 = np.random.normal(1, 2, n)
eps = np.random.normal(0.0, 0.1, n)
w = np.array([0.2,0.5,-0.25])
X = np.transpose(np.array([x1,x2,x3]))
print(w)
print(X)
Y = X.dot(w) + eps
Y = Y.reshape(n,1)
print(Y)

[ 0.2   0.5  -0.25]
[[-0.1542462  -1.71387555 -3.10841518]
 [-1.28812839 -2.02166673 -1.55597569]
 [-0.13611584  0.22465953 -0.43794157]
 [-0.72921205  0.74184753  2.48140871]
 [-1.01964645  0.48577868  0.48729923]
 [ 1.36515182  1.46765055  1.69898819]
 [ 1.9350056  -0.04091108 -1.85659773]
 [-0.90823159  0.47379474  2.33687514]
 [ 0.49212619 -1.17373707 -2.16787315]
 [-0.1819442  -0.69115747 -2.39002991]]
[[-0.0860665 ]
 [-0.98170197]
 [ 0.06027654]
 [-0.44957531]
 [-0.13596702]
 [ 0.53037306]
 [ 0.82036704]
 [-0.42570698]
 [ 0.09189029]
 [ 0.19882579]]


## Gradient descent 

### Problem 1 

Find the gradient of the loss function with respect to weights.

Write gradient function ``grad(y,x,w)``.

In [70]:
def grad(x,y,w):
    return np.array((
        np.sum( derivative_x(x,y,w)),
        np.sum(derivative_y(x,y,w))
    ))/(2*len(x))
    
def derivative_x(x, y, w):
    return -2*y*w+2*x*np.power(w,2)
def derivative_y(x, y, w):
    return 2*y-2*x*w
# def derivative_w(x, y, w):
#     return -2*y*w+2*pow(x,2)*w


### Problem 2

Implement gradient descent for linear regression.

In [77]:
def gradient_descent(X, y, me, ce, learning_rate=0.000001):
    y_pred = me*X + ce  # The current predicted value of Y
    d_m = (-2/n) * sum(X * (y - y_pred))  # Derivative wrt m
    d_c = (-2/n) * sum(y - y_pred)  # Derivative wrt c
    me = me - learning_rate * d_m  # Update m
    ce = ce - learning_rate * d_c  # Update c
    return me, ce

#CORE
max_epoch = 500
m = 0
c = 0

for epoch in range(1,max_epoch+1):
    m,c = gradient_descent(X, Y, m, c)
    if epoch%100 ==0:
        print(epoch)
        print (m, c)

100
[ 8.88560636e-05  4.08754259e-05 -3.40750195e-05] [-7.54440382e-06 -7.54404430e-06 -7.54647610e-06]
200
[ 1.77694297e-04  8.17406248e-05 -6.81228738e-05] [-1.50861880e-05 -1.50847429e-05 -1.50945166e-05]
300
[ 0.00026651  0.0001226  -0.00010214] [-2.26253533e-05 -2.26220969e-05 -2.26441189e-05]
400
[ 0.00035532  0.00016344 -0.00013614] [-3.01619004e-05 -3.01561075e-05 -3.01952800e-05]
500
[ 0.0004441   0.00020427 -0.0001701 ] [-3.76958302e-05 -3.76867758e-05 -3.77479973e-05]


### Problem 3

Implement stochastic gradient descent (SGD).

In [78]:
max_epoch = 10000
m, c = 0,0

for epoch in range(1,max_epoch+1):
    sample = np.random.randint(0,n)
    x = X[sample]
    y= Y[sample]
    m,c = gradient_descent(x, y, m, c, 0.00001)
    # if epoch % 1000 == 1:
    #     print(f'y = {w_s:.2f} x + {b_s:.2f}')
        
print(f'y = {m:.2f} x + {c:.2f}')

y = 0.01 x + -0.00


### Problem 4

Implement SGD using pytorch. Start by just rewritting Problem 3 to use torch Tensors instead of numpy arrays. 

To convert frrom numpy arrays to torch tensors you can use ``torch.from_numpy()`` function. 

In [79]:
def gradient_descent_tensor(X, y, w, b, learning_rate=0.000001):
    dw = -2 * torch.sum(X * (y - w * X - b)) # ∂e/∂w
    db = -2 * torch.sum(y - w * X - b)       # ∂e/∂b
    w_new = w - learning_rate * dw        # minus sign since we are minizing e
    b_new = b - learning_rate * db
    return w_new, b_new

x_tensor = torch.from_numpy(X).float()
y_tensor = torch.from_numpy(Y).float()
torch.manual_seed(42)
n_epochs = 1000
w_t = torch.randn(1, requires_grad=False, dtype=torch.float)
b_t = torch.randn(1, requires_grad=False, dtype=torch.float)
for epoch in range(1,n_epochs+1):
    w_t,b_t = gradient_descent_tensor(x_tensor, y_tensor, w_t, b_t)

print(w_t, b_t)

tensor([0.3076]) tensor([0.1237])


### Problem 5 

Implement GD using pytorch automatic differentiation.

To this end the variable with respect to which the gradient will be calculated, ``t_w`` in this case, must have attribute
``requires_grad`` set to ``True`` (``t_w.require_grad=True``).

The torch will automatically track any expression containing ``t_w`` and store its computational graph. The method ``backward()`` can be run on the final expression to back propagate the gradient e.g. ``loss.backward()``. Then the gradient is accesible as ``t_w.grad``.

In [74]:
import torch.optim as optim
import torch.nn as nn

In [80]:
torch.manual_seed(42)
n_epochs = 1000
learning = 0.01
a = torch.randn(1, requires_grad=True, dtype=torch.float)
b = torch.randn(1, requires_grad=True, dtype=torch.float)
loss_fn = nn.MSELoss(reduction='mean')
optimizer = optim.SGD([a,b], lr=learning)
for epoch in range(1,n_epochs+1):
    yhat = a + b * x_tensor
    # error = y_tensor - yhat
    # loss = (error ** 2).mean() 
    loss = loss_fn(y_tensor,yhat)
    loss.backward()
    # with torch.no_grad():
    #     a -= lr * a.grad
    #     b -= lr * b.grad
    optimizer.step()
    # a.grad.zero_()
    # b.grad.zero_()
    optimizer.zero_grad()
print(a,b)

C:\Users\Kamil\Anaconda3\lib\site-packages\torch\nn\modules\loss.py:431: UserWarning: Using a target size (torch.Size([10, 3])) that is different to the input size (torch.Size([10, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


tensor([-0.0196], requires_grad=True) tensor([0.0737], requires_grad=True)
